# Twitter API

**API Documentation: ** https://dev.twitter.com/rest/public

## Setting Up

In [1]:
import json
import twitter

In [2]:
def read_keys(path):
    with open(path, 'r') as f:
        keys = map(lambda s: s.strip(), f.readlines())
    return keys


KEYS_PATH = './oauth_twitter.keys.txt'
CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET = \
    read_keys(KEYS_PATH)
    
auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                          CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

print twitter_api

## Exploring trends

To get codes: https://developer.yahoo.com/geo/geoplanet/

ID=1 - for the entire world

ID=2342977 -  for the US

In [3]:
# Receiving World trends and US trends
WORLD_WOE_ID = 1
US_WOE_ID = 23424977

world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
us_trends = twitter_api.trends.place(_id=US_WOE_ID)

print(json.dumps(world_trends, indent=1))
print('===')
print(json.dumps(us_trends, indent=1))

[
 {
  "created_at": "2016-08-16T19:26:12Z", 
  "trends": [
   {
    "url": "http://twitter.com/search?q=%23FutebolFeminino", 
    "query": "%23FutebolFeminino", 
    "tweet_volume": 198188, 
    "name": "#FutebolFeminino", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=Su%C3%A9cia", 
    "query": "Su%C3%A9cia", 
    "tweet_volume": 71407, 
    "name": "Su\u00e9cia", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%237FavTVShows", 
    "query": "%237FavTVShows", 
    "tweet_volume": 60885, 
    "name": "#7FavTVShows", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23%D8%A7%D9%84%D9%86%D8%B5%D8%B1_%D8%A7%D9%84%D9%86%D8%AC%D9%88%D9%85", 
    "query": "%23%D8%A7%D9%84%D9%86%D8%B5%D8%B1_%D8%A7%D9%84%D9%86%D8%AC%D9%88%D9%85", 
    "tweet_volume": 59329, 
    "name": "#\u0627\u0644\u0646\u0635\u0631_\u0627\u0644\u0646\u062c\u0648\u0645", 
    "promoted_content": null
   }, 
   {
    "url"

In [4]:
# Intersection of two sets of trends

get_trends = lambda it: set([trend['name'] for trend in it[0]['trends']])

world_trends_set = get_trends(world_trends)
us_trends_set = get_trends(us_trends)

common_trends = world_trends_set.intersection(us_trends_set)

print(common_trends)

set([u'Aguero', u'#JustinYoureNotAlone', u'Amy Tinkler', u'#7FavTVShows', u'FIFA 17', u'#WinMeOverIn4Words', u'#VoteTheVampsR1', u'#RomanticElectionSongs', u'#NationalTellAJokeDay'])


## Searching for tweets

https://dev.twitter.com/rest/reference/get/search/tweets

In [17]:
q = '#RomanticElectionSongs'

count = 100

search_results = twitter_api.search.tweets(q=q, count=count)
statuses = search_results['statuses']

for __ in range(5):
    print('Length of statuses {}'.format(len(statuses)))
    try:
        next_results = search_results['search_metadata']['next_result']
    except KeyError, e:
        break
    
    kwargs = dict([kv.split('=') for kv in next_results[1:].split('&')])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']
    
print(json.dumps(statuses[0], indent=1))

Length of statuses 100
{
 "contributors": null, 
 "truncated": false, 
 "text": "RT @BrutusWatts: #RomanticElectionSongs  Hello..Is it me you're voting for?", 
 "is_quote_status": false, 
 "in_reply_to_status_id": null, 
 "id": 765650394643046400, 
 "favorite_count": 0, 
 "entities": {
  "symbols": [], 
  "user_mentions": [
   {
    "id": 1634948832, 
    "indices": [
     3, 
     15
    ], 
    "id_str": "1634948832", 
    "screen_name": "BrutusWatts", 
    "name": "Chris Watts"
   }
  ], 
  "hashtags": [
   {
    "indices": [
     17, 
     39
    ], 
    "text": "RomanticElectionSongs"
   }
  ], 
  "urls": []
 }, 
 "retweeted": false, 
 "coordinates": null, 
 "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", 
 "in_reply_to_screen_name": null, 
 "in_reply_to_user_id": null, 
 "retweet_count": 5, 
 "id_str": "765650394643046400", 
 "favorited": false, 
 "retweeted_status": {
  "contributors": null, 
  "truncated": false, 
  "text": "#RomanticElection

In [18]:
print(json.dumps(search_results['search_metadata'], indent=1))

{
 "count": 100, 
 "completed_in": 0.142, 
 "max_id_str": "765650394643046400", 
 "since_id_str": "0", 
 "next_results": "?max_id=765647967218987007&q=%23RomanticElectionSongs&count=100&include_entities=1", 
 "refresh_url": "?since_id=765650394643046400&q=%23RomanticElectionSongs&include_entities=1", 
 "since_id": 0, 
 "query": "%23RomanticElectionSongs", 
 "max_id": 765650394643046400
}


## Extracting Tweet Entities

In [19]:
status_texts = [status['text'] for status in statuses]

screen_names = [user_mention['screen_name'] for status in statuses
                  for user_mention in status['entities']['user_mentions']]

hashtags = [hashtag['text'] for status in statuses
               for hashtag in status['entities']['hashtags']]

# Computee a collection of all words from all tweets
words = [w for t in status_texts
            for w in t.split()]

print('Statuses: ')
print(json.dumps(status_texts[0:5], indent=1))
print('\nScreen names: ')
print(json.dumps(screen_names[0:5], indent=1))
print('\nHashtags: ')
print(json.dumps(hashtags[0:5], indent=1))
print('\nWords: ')
print(json.dumps(words[0:5], indent=1))

Statuses: 
[
 "RT @BrutusWatts: #RomanticElectionSongs  Hello..Is it me you're voting for?", 
 "RT @___0_0_0_0__: #RomanticElectionSongs Same song different dance Recycling bullshit from 2008 https://t.co/ZKNQb4CqhJ", 
 "RT @larryhedgehog1: #RomanticElectionSongs \"Too much time on my tiny hands\"", 
 "RT @MissLizzyNJ: #RomanticElectionSongs\nAin't Too Proud To Beg https://t.co/kkDnKIu2f5", 
 "RT @___0_0_0_0__: #RomanticElectionSongs Same song different dance Recycling bullshit from 2008 https://t.co/ZKNQb4CqhJ"
]

Screen names: 
[
 "BrutusWatts", 
 "___0_0_0_0__", 
 "larryhedgehog1", 
 "MissLizzyNJ", 
 "___0_0_0_0__"
]

Hashtags: 
[
 "RomanticElectionSongs", 
 "RomanticElectionSongs", 
 "RomanticElectionSongs", 
 "RomanticElectionSongs", 
 "RomanticElectionSongs"
]

Words: 
[
 "RT", 
 "@BrutusWatts:", 
 "#RomanticElectionSongs", 
 "Hello..Is", 
 "it"
]


## Frequency analysis